In [5]:
!pip3 install imblearn

    100% |████████████████████████████████| 174kB 3.1MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/cd/c1/50a758e8247561e58cb87305b1e90b171b8c767b15b12a1734001f41d356/joblib-0.13.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/b9/bda9781f0a74b90ebd2e046fde1196182900bd4a8e1ea503d3ffebc50e7c/numpy-1.17.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a0/c5/d2238762d780dde84a20b8c761f563fe882b88c5a5fb03c056547c442a19/scikit_learn-0.21.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/29/50/a552a5aff252ae915f522e44642bb49a7b7b31677f9580cfd11bcc869976/scipy-1.3.1-cp36-cp36m-manylinux1_x86_64.whl


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pandas_profiling

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

import xgboost as xgb

from datetime import datetime
import lightgbm as lgb
#import multiprocessing
#import gc


In [33]:
from sklearn.preprocessing import MinMaxScaler  
##smote 데이터 스케일링

In [34]:
#!pip3 install imblearn

In [35]:
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE ##SMOTE 오예~ 

In [37]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [38]:
import os
import gc
import sys

In [39]:
trans = pd.read_csv('../../Data/original/train_transaction.csv', index_col='TransactionID')
test_trans = pd.read_csv('../../Data/original/test_transaction.csv', index_col='TransactionID')

In [40]:
ident = pd.read_csv('../../Data/original/train_identity.csv', index_col='TransactionID')
test_ident = pd.read_csv('../../Data/original/test_identity.csv', index_col='TransactionID')

In [41]:
sample_sub = pd.read_csv('../../Data/original/sample_submission.csv', index_col='TransactionID')

In [42]:
train = trans.merge(ident, how='left', left_index=True, right_index=True)
test = test_trans.merge(test_ident, how='left', left_index=True, right_index=True)

In [43]:
del trans, ident, test_trans, test_ident
gc.collect()

0

In [44]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 668.22 Mb (66.2% reduction)
Mem. usage decreased to 583.43 Mb (65.6% reduction)


In [45]:
final_cols = ['isFraud','TransactionAmt','TransactionDT',
 'P_emaildomain',
 'R_emaildomain', "ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", 
"addr1", "addr2", "dist1", "dist2", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", 
"C11", "C12", "C13", "C14", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12", 
"D13", "D14", "D15", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "id_01", "id_02", 
"id_03", "id_04", "id_05", "id_06", "id_07", "id_08", "id_09", "id_10", "id_11", 
"id_12", "id_13", "id_14", "id_15", "id_16", "id_17", "id_18", "id_19", "id_20", "id_21", "id_22", "id_23", "id_24", 
"id_25", "id_26", "id_27", "id_28", "id_29", "id_30", "id_31", "id_32", "id_33", "id_34", "id_35", 
"id_36", "id_37", "id_38", 
'V1','V2','V3','V4','V5','V6','V7','V9','V12','V13','V17','V18','V19',
'V20','V23','V24','V25','V26','V29','V30','V33',
 'V34','V35','V36','V37','V38','V39','V40','V42','V43','V44','V45','V47','V48','V49','V51','V52','V53','V54','V55',
 'V56','V57','V58','V59','V60','V61','V62','V63','V64','V66','V67','V69','V70','V71','V72','V73','V74','V75','V76',
 'V77','V78','V79','V80','V81','V82','V83','V85','V86','V87','V90','V91','V92','V93','V94', 'V95', 'V96', 'V97',
 'V98','V99','V100','V101','V102','V103','V105','V109','V115','V124','V125','V126','V127','V128','V129', 'V130','V131',
 'V132', 'V133', 'V134','V135','V136','V137','V139','V140','V143','V145','V146','V147','V149','V150','V151','V152',
 'V156','V158','V159','V160','V161','V162','V164','V165','V166',
 'V167','V168','V169','V170','V171','V172','V173','V174','V175','V176','V177','V178','V179','V180','V182','V187','V188',
 'V189','V192','V198','V199','V200','V201','V202','V203','V204','V205','V206','V207','V208','V209','V210',
 'V211', 'V212','V213','V214','V215','V216','V217','V218','V219','V220','V221','V222','V223','V224','V225','V226',
 'V227','V228','V229','V230','V231','V232','V233','V234','V236','V239','V243','V244','V245','V246','V248','V250',
 'V251','V256','V257',
 'V258','V259','V260','V261','V262','V263','V264','V265','V266','V267','V268','V270','V271','V272','V273','V274',
 'V275','V276','V277','V278','V279','V280','V281','V282','V283','V284','V285','V286','V287','V288','V289','V290',
 'V291','V292','V293','V294','V295',
 'V296','V297','V298','V299','V300','V301','V303','V304','V306','V307','V308','V309','V310','V311','V312',
 'V313','V314','V315','V316','V317','V318','V319','V320','V321','V322','V323','V324','V326','V329','V331',
 'V332','V333','V335','V336','V338','V339']

In [46]:

del_cols = []
no_cols= []
for col in train.columns:

  if col not in final_cols:
    del_cols.append(col)
  else:
    no_cols.append(col)
print(len(no_cols));print(len(del_cols))
print(no_cols == final_cols)

351
82
False


In [47]:
print(len(del_cols))
del_cols

82


['V8',
 'V10',
 'V11',
 'V14',
 'V15',
 'V16',
 'V21',
 'V22',
 'V27',
 'V28',
 'V31',
 'V32',
 'V41',
 'V46',
 'V50',
 'V65',
 'V68',
 'V84',
 'V88',
 'V89',
 'V104',
 'V106',
 'V107',
 'V108',
 'V110',
 'V111',
 'V112',
 'V113',
 'V114',
 'V116',
 'V117',
 'V118',
 'V119',
 'V120',
 'V121',
 'V122',
 'V123',
 'V138',
 'V141',
 'V142',
 'V144',
 'V148',
 'V153',
 'V154',
 'V155',
 'V157',
 'V163',
 'V181',
 'V183',
 'V184',
 'V185',
 'V186',
 'V190',
 'V191',
 'V193',
 'V194',
 'V195',
 'V196',
 'V197',
 'V235',
 'V237',
 'V238',
 'V240',
 'V241',
 'V242',
 'V247',
 'V249',
 'V252',
 'V253',
 'V254',
 'V255',
 'V269',
 'V302',
 'V305',
 'V325',
 'V327',
 'V328',
 'V330',
 'V334',
 'V337',
 'DeviceType',
 'DeviceInfo']

In [48]:
train = train.drop(del_cols, axis=1)
test = test.drop(del_cols, axis=1)

In [49]:
for column in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2']:
    train[column +  '_count_full'] = \
        train[column].map(pd.concat([train[column], test[column]], ignore_index=True).value_counts(dropna=False))
    test[column +  '_count_full'] = \
        test[column].map(pd.concat([train[column], test[column]], ignore_index=True).value_counts(dropna=False))

In [50]:
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

In [51]:
train.columns

Index(['isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1',
       'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'card3_count_full', 'card4_count_full', 'card5_count_full',
       'card6_count_full', 'addr1_count_full', 'addr2_count_full',
       'TransactionAmt_to_mean_card1', 'TransactionAmt_to_mean_card4',
       'TransactionAmt_to_std_card1', 'TransactionAmt_to_std_card4'],
      dtype='object', length=363)

In [52]:
# 준형이형꺼 처리
#id_30

def os(df):
    if df == "-999":
        df = -999
    elif "Windows" in df:
        df = "Windows"
    elif "Mac" in df:
        df = "Mac"
    elif "iOS" in df:
        df = "ios"
    elif "Android" in df:
        df = "Android"
    else:
        df = "other"
        
    return df

train["id_30"].fillna("-999", inplace=True)
train["OS"] = train["id_30"].apply(os)

test["id_30"].fillna("-999", inplace=True)
test["OS"] = test["id_30"].apply(os)

#id_31 
# latest 여부 / 브라우저 col 생성
latest_browser_list = ["samsung browser 7.0",
                  "opera 53.0",
                  "mobile safari 10.0",
                  "google search application 49.0",
                  "firefox 60.0",
                  "edge 17.0",
                  "chrome 69.0", 
                  "chrome 67.0 for android",
                  "chrome 63.0 for android",
                  "chrome 63.0 for ios",
                  "chrome 64.0",
                  "chrome 64.0 for android",
                  "chrome 64.0 for ios",
                  "chrome 65.0",
                  "chrome 65.0 for android",
                  "chrome 65.0 for ios",
                  "chrome 66.0",
                  "chrome 66.0 for android",
                  "chrome 66.0 for ios"
                 ]

def latest_browser(df):
    if df == "-999":
        df = -999
    elif df in latest_browser_list:
        df = 1
    else:
        df = 0
    return df

def browser(df):
    if df == "-999":
        df = -999
    elif "chrome" in df:
        df = "chrome"
    elif "safari" in df:
        df = "safari"
    elif "firefox" in df:
        df = "firefox"
    elif "ie" in df:
        df = "ie"
    elif "samsung" in df:
        df = "samsumg"
    elif "Samsung" in df:
        df = "samsumg"
    return df 
    

train["id_31"].fillna("-999", inplace=True)
train["Latest_browser"] = train["id_31"].apply(latest_browser)
train["Browser"] = train["id_31"].apply(browser)

test["id_31"].fillna("-999", inplace=True)
test["Latest_browser"] = test["id_31"].apply(latest_browser)
test["Browser"] = test["id_31"].apply(browser)

In [53]:
y_train = train['isFraud'].copy()

In [54]:
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

In [55]:
# Fill numeric columns' NaN values with mean
for c in X_train.columns:
    if X_train[c].dtype=='float16' or  X_train[c].dtype=='float32' or  X_train[c].dtype=='float64':
        X_train[c].fillna(X_train[c].mean())
        X_test[c].fillna(X_train[c].mean())

In [56]:

# Fill categorical columns' NaN values with placeholder (-999)
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [57]:
# Label-Encode categorical columns
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))

In [36]:
f =1

In [59]:
X_train_ = X_train.copy()
y_train_ = y_train.copy()
X_train_[X_train_ == np.inf] = np.nan
y_train_[y_train_ == np.inf] = np.nan
X_train_ = X_train_.fillna(-999)
y_train_ = y_train_.fillna(-999)

In [61]:
# 모델설정
sm = SMOTE(ratio='auto', kind='regular')

# train데이터를 넣어 복제함
X_resampled, y_resampled = sm.fit_sample(X_train_,list(y_train_))

print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))

After OverSampling, the shape of train_X: (1139754, 365)
After OverSampling, the shape of train_y: (1139754, 365) 

After OverSampling, counts of label '1': 569877
After OverSampling, counts of label '0': 569877


In [72]:
type(X_resampled)

numpy.ndarray

In [62]:
# Set up K-Fold
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)

print(folds)

KFold(n_splits=5, random_state=None, shuffle=True)


In [41]:
# train.to_csv('smote_train.csv')
# test.to_csv('smote_test.csv')

In [73]:
X_train.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,card6_count_full,addr1_count_full,addr2_count_full,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card4,TransactionAmt_to_std_card1,TransactionAmt_to_std_card4,OS,Latest_browser,Browser
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,86400,68.5,4,13926,-999.0,150.0,2,142.0,2,315.0,...,267648,43035,956415,0.194580,0.257812,0.000000,0.000000,0,-999,0
2987001,86401,29.0,4,2755,404.0,150.0,3,102.0,2,325.0,...,267648,76902,956415,0.123779,0.219116,0.000000,0.114258,0,-999,0
2987002,86469,59.0,4,4663,490.0,150.0,4,166.0,3,330.0,...,824959,48387,956415,0.608398,0.443115,0.589355,0.258545,0,-999,0
2987003,86499,50.0,4,18132,567.0,150.0,3,117.0,3,476.0,...,824959,17455,956415,0.405029,0.377686,0.259521,0.196899,0,-999,0
2987004,86506,50.0,1,4497,514.0,150.0,3,102.0,2,420.0,...,267648,7107,956415,0.515625,0.377686,0.882812,0.196899,1,0,64


In [77]:
type(X_resampled)

numpy.ndarray

In [78]:
pd.DataFrame(X_resampled).head()

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
0,86400.0,68.5,4.0,13926.0,-999.0,150.0,2.0,142.0,2.0,315.0,...,267648.0,43035.0,956415.0,0.194580,0.257812,0.000000,0.000000,0.0,-999.0,0.0
1,86401.0,29.0,4.0,2755.0,404.0,150.0,3.0,102.0,2.0,325.0,...,267648.0,76902.0,956415.0,0.123779,0.219116,0.000000,0.114258,0.0,-999.0,0.0
2,86469.0,59.0,4.0,4663.0,490.0,150.0,4.0,166.0,3.0,330.0,...,824959.0,48387.0,956415.0,0.608398,0.443115,0.589355,0.258545,0.0,-999.0,0.0
3,86499.0,50.0,4.0,18132.0,567.0,150.0,3.0,117.0,3.0,476.0,...,824959.0,17455.0,956415.0,0.405029,0.377686,0.259521,0.196899,0.0,-999.0,0.0
4,86506.0,50.0,1.0,4497.0,514.0,150.0,3.0,102.0,2.0,420.0,...,267648.0,7107.0,956415.0,0.515625,0.377686,0.882812,0.196899,1.0,0.0,64.0


In [75]:
X_resampled.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [64]:
pwd

'/home/etture2/ybigta_first_proj/Models'

In [65]:
# Initialize submission DataFrame
sample_submission = pd.read_csv('../../Data/original/sample_submission.csv')
lgb_submission = sample_submission.copy()
lgb_submission['isFraud'] = 0


In [67]:
import time

In [81]:
# If using GPU, set this option to True
gpu_enabled = False

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'n_estimators': 1000,
    'colsample_bytree': 0.85,
    'subsample_freq': 1,
    'subsample': 0.85,
    'max_bin': 255,
    'metric': 'auc',
    'n_jobs': -1,
    'num_leaves': 2**8,
    'max_depth': 10,
    'tree_learner': 'serial',
    'learning_rate': 0.05,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'reg_alpha': 0.3,
    'reg_lambda': 0.243
}

if gpu_enabled:
    params.update({
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0
    })
    
params['learning_rate'] = 0.02
params['max_depth'] = 8
params['num_leaves'] = 128
params['colsample_bytree'] = 0.7
params['subsample'] = 0.9
params['reg_alpha'] = 0.45
params['reg_lambda'] = 0.4

X_resampled_ = pd.DataFrame(X_resampled)
y_resampled_ = pd.DataFrame(y_resampled)
# for fold_n, (train_index, valid_index) in enumerate(folds.split(X_resampled)):    
for fold_n, (train_index, valid_index) in enumerate(folds.split(X_resampled_)):    
    # Print current iteration information
    start = time.time()
    now = time.localtime(start)
    print('Fold {} started..., Time: {yy}-{mm}-{dd} {hh}:{MM}:{ss}'.format(fold_n, yy=now.tm_year, mm=now.tm_mon, dd=now.tm_mday, hh=now.tm_hour, MM=now.tm_min, ss=now.tm_sec))
    
    X_train_, X_valid = X_resampled_.iloc[train_index], X_resampled_.iloc[valid_index]
    y_train_, y_valid = y_resampled_.iloc[train_index], y_resampled_.iloc[valid_index]
    
    lgb_train = lgb.Dataset(X_train_, label=y_train_)
    lgb_test = lgb.Dataset(X_valid, label=y_valid)
    
    lgb_estimator = lgb.train(
        params,
        lgb_train,
        valid_sets = [lgb_test],
        verbose_eval=100,
        early_stopping_rounds=50
    )
    
    del X_train_, y_train_
    
    pred = lgb_estimator.predict(X_test)
    val = lgb_estimator.predict(X_valid)
    
    # Measure time elapsed
    now = time.time()
    time_elapsed = now - start
    print('Completed, Time elapsed: {} seconds'.format(time_elapsed))
    
    print('ROC accuracy: {}'.format(roc_auc_score(y_valid, val)))
    
    del val, y_valid
    
    lgb_submission['isFraud'] = lgb_submission['isFraud'] + pred/n_fold
    
    del pred
    gc.collect()
    
    print('-------------------------------------------------------')
    
print('Done.')

Fold 0 started..., Time: 2019-8-24 7:7:20
Training until validation scores don't improve for 50 rounds.
[100]	valid_0's auc: 0.991511
[200]	valid_0's auc: 0.995394
[300]	valid_0's auc: 0.996467
[400]	valid_0's auc: 0.996987
[500]	valid_0's auc: 0.99727
[600]	valid_0's auc: 0.997506
[700]	valid_0's auc: 0.997724
[800]	valid_0's auc: 0.997887
[900]	valid_0's auc: 0.998033
[1000]	valid_0's auc: 0.998165
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.998165
Completed, Time elapsed: 262.58616161346436 seconds
ROC accuracy: 0.998165004087519
-------------------------------------------------------
Fold 1 started..., Time: 2019-8-24 7:11:43


/home/etture2/.local/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[100]	valid_0's auc: 0.991786
[200]	valid_0's auc: 0.995453
[300]	valid_0's auc: 0.996509
[400]	valid_0's auc: 0.997014
[500]	valid_0's auc: 0.997301
[600]	valid_0's auc: 0.997558
[700]	valid_0's auc: 0.99775
[800]	valid_0's auc: 0.997907
[900]	valid_0's auc: 0.998058
[1000]	valid_0's auc: 0.99818
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.99818
Completed, Time elapsed: 262.4127674102783 seconds
ROC accuracy: 0.9981800555666802
-------------------------------------------------------
Fold 2 started..., Time: 2019-8-24 7:16:6


/home/etture2/.local/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[100]	valid_0's auc: 0.991644
[200]	valid_0's auc: 0.995292
[300]	valid_0's auc: 0.996355
[400]	valid_0's auc: 0.996849
[500]	valid_0's auc: 0.997187
[600]	valid_0's auc: 0.997413
[700]	valid_0's auc: 0.997587
[800]	valid_0's auc: 0.997748
[900]	valid_0's auc: 0.997917
[1000]	valid_0's auc: 0.998072
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.998072
Completed, Time elapsed: 266.0423758029938 seconds
ROC accuracy: 0.9980723293135798
-------------------------------------------------------
Fold 3 started..., Time: 2019-8-24 7:20:32


/home/etture2/.local/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[100]	valid_0's auc: 0.991651
[200]	valid_0's auc: 0.995318
[300]	valid_0's auc: 0.996439
[400]	valid_0's auc: 0.996962
[500]	valid_0's auc: 0.997285
[600]	valid_0's auc: 0.997505
[700]	valid_0's auc: 0.997717
[800]	valid_0's auc: 0.99788
[900]	valid_0's auc: 0.998028
[1000]	valid_0's auc: 0.998158
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.998158
Completed, Time elapsed: 259.89632201194763 seconds
ROC accuracy: 0.9981575069900496
-------------------------------------------------------
Fold 4 started..., Time: 2019-8-24 7:24:52


/home/etture2/.local/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[100]	valid_0's auc: 0.991788
[200]	valid_0's auc: 0.99531
[300]	valid_0's auc: 0.996412
[400]	valid_0's auc: 0.996906
[500]	valid_0's auc: 0.997219
[600]	valid_0's auc: 0.997446
[700]	valid_0's auc: 0.997655
[800]	valid_0's auc: 0.997815
[900]	valid_0's auc: 0.997944
[1000]	valid_0's auc: 0.998078
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.998078
Completed, Time elapsed: 260.17385149002075 seconds
ROC accuracy: 0.9980782373135
-------------------------------------------------------
Done.


In [85]:
pwd

'/home/etture2/ybigta_first_proj/Models'

In [86]:
dir_name = 'submission'
lgb_submission.to_csv('./{directory}/sub_lightgbm_smote.csv'.format(directory=dir_name), index=False)
lgb_submission.head()

,TransactionID,isFraud
0,3663549,0.004235
1,3663550,0.006467
2,3663551,0.004909
3,3663552,0.004498
4,3663553,0.007498
